In [1]:
import tensorflow as tf

In [2]:
import os
os.chdir("../")

In [ ]:
# MLFLOW_TRACKING_URI=https://dagshub.com/shaik-sys/DeepLearningCnnClassifier.mlflow \
# MLFLOW_TRACKING_USERNAME=shaik-sys \
# MLFLOW_TRACKING_PASSWORD=97db02a7c313c17bc8ffc0ed12bc9eb98a11525b  \
# python script.py

In [3]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/shaik-sys/DeepLearningCnnClassifier.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "shaik-sys"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "97db02a7c313c17bc8ffc0ed12bc9eb98a11525b"

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model = Path
    training_data: Path
    all_params : dict
    mlflow_uri:str
    params_image_size :list
    params_batch_size: int

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int


In [27]:
from DeepClassifier.constants import *
from DeepClassifier.utils import read_yaml, create_directories

In [28]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])



    def get_validation_config(self) -> EvaluationConfig:
        evaluation_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/PetImages",
            all_params= self.params,
            mlflow_uri="https://dagshub.com/shaik-sys/DeepLearningCnnClassifier.mlflow",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return evaluation_config

    

In [29]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [30]:
from DeepClassifier.utils.common import save_json

In [31]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)



    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [34]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()
    
except Exception as e:
   raise e

Found 7498 images belonging to 2 classes.
469/469 [==============================] - 2123s 5s/step - loss: 1.7584 - accuracy: 0.8545
INFO:tensorflow:Assets written to: C:\Users\shaai\AppData\Local\Temp\tmpa3avhve7\model\data\model\assets


AttributeError: module 'tensorflow.keras' has no attribute '__version__'

In [33]:
tf.__version__

'2.13.0'

In [35]:
evaluation.log_into_mlflow()

INFO:tensorflow:Assets written to: C:\Users\shaai\AppData\Local\Temp\tmpgx55moli\model\data\model\assets


AttributeError: module 'tensorflow.keras' has no attribute '__version__'